In [ ]:
import ee

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
import geemap

In [ ]:
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)


img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

PreFire = ee.Filter.date('2021-06-01', '2021-07-08')
PostFirePreSlide = ee.Filter.date('2022-06-01', '2022-07-03')
PostSlide = ee.Filter.date('2022-07-05', '2022-09-30')

desc_change = ee.Image.cat(
    desc.filter(PreFire).mean(),
    desc.filter(PostFirePreSlide).mean(),
    desc.filter(PostSlide).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(PreFire).mean(),
    asc.filter(PostFirePreSlide).mean(),
    asc.filter(PostSlide).mean(),
)

m = geemap.Map()
m.set_center(48.5,-120.2, 6)
m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m.add_geojson('StudyArea.geojson')

m
